## Testing the sequential segmentation algorithm with simulated data


In [ ]:
%matplotlib inline
import sys
sys.path.append('/home/paulo/github')

import time
import datetime

import numpy as np
import pandas as pd

from SeqSeg.SeqSeg import SeqSeg

savefolder = '/home/paulo/github/bayeseg/Output/'

## Computing time for various time resolutions

In [ ]:
# This comparison cannot be made to be exact with the results in the paper, since it depends 
# on the system's characteristics and on other software that might be possibly running simultaneously

ss = SeqSeg()


sizes = [10000, 100000, 1000000]
tres = [1, 10, 100, 1000]

ss.initialize(1, 0.1, 100, 200, 1)
result = []
np.random.seed(123456)
for size in sizes:
    signal = np.random.normal(0, 1, [size, 1])
    ss.feed_data(signal)
    for res in tres:
        t, tdur = ss.segments(minlen = 1000000, res = res, verbose = False)
        print("Size = " + str(size) + ", tres = " + str(res) + ", t = ", str(tdur))
        result.append([size, res, tdur])

## Testing $\alpha$ and $\beta$ with many segments

In [ ]:
npoints = 1000000
cuts = [10000, 110000, 200000, 500000, 750000, 1000000]
deltalist = [1, 1.1, 1.5]
stdnoise = 1

mciter = 10000
mcburn = 10000
alphalist = [0.1, 0.5, 0.9, 0.99]
betalist = [1, 1e-1, 1e-2, 1e-3, 2e-3, 5e-3, 1e-4, 5e-4, 1e-5, 1e-6]

nruns = 30

ss = SeqSeg()
res = []
np.random.seed(123456)
for delta in deltalist:
    # Simulates signal
    signal = np.random.normal(0, 1, [npoints, 1])
    for i in range(len(cuts)):
        if (i+1)%2:
            signal[cuts[i]:cuts[i+1]] = signal[cuts[i]:cuts[i+1]]*(np.sqrt(delta))
    ss.feed_data(signal)
    for beta in betalist:
        for alpha in alphalist:
            ss.initialize(beta, alpha, mciter, mcburn, nchains = 1)
            nsegmean = 0
            minseg = 500000
            maxseg = 0
            tmean = 0
            for i in range(nruns):
                t, tdur = ss.segments(minlen = 5000, res = 100, verbose = False)
                nsegmean = nsegmean + (len(t)+1)/nruns
                tmean = tmean + tdur / nruns
                if len(t) + 1 < minseg:
                    minseg = len(t) + 1
                if len(t) + 1 > maxseg:
                    maxseg = len(t) + 1
            
            print("Delta = " + str(delta) + ", alpha = " + str(alpha) + ", beta = " + str(beta) + ", " + str(nsegmean) + " segments on average in " + "{:.2}".format(tmean) + " seconds.")
                
            res.append([delta, beta, alpha, nsegmean, minseg, maxseg, tdur])
            

df = pd.DataFrame(res, columns = ['delta', 'beta', 'alpha', 'nsegmean', 'minseg', 'maxseg', 'tmean'])

## Calibration for a grid of $\beta$ and $\alpha$ with varying delta

In [ ]:
# Signal characteristics
npoints = 1000000
cuts = [10000, 110000, 200000, 500000, 750000, 1000000]
delta = [1.1, 1, 1.5, 1, 1.2]
stdnoise = 1

mciter = 10000
mcburn = 10000

# Alpha grid
alphamin = 0.1
alphamax = 0.1
nstep = 0
adelta = (alphamax - alphamin) / max(1, nstep)
alphalist = [alphamin + d*adelta for d in range(nstep+1)]

# Beta grid
betamin = 1e-3
betamax = 1e-1
nstep = 100
bdelta = (betamax - betamin) / max(1, nstep)
betalist = [betamin + d*bdelta for d in range(nstep+1)]

# To use union of two separate grids 
#betamin = 1e-3
#betamax = 1e-1
#nstep = 100
#bdelta = (betamax - betamin) / max(1, nstep)
#betalist = betalist + [betamin + d*bdelta for d in range(nstep+1)]

# Number of segmentations to run for each combination of parameters
nruns = 1

ss = SeqSeg()
res = []

# Simulates signal
signal = np.random.normal(0, stdnoise, [npoints, 1])
for i in range(len(cuts)-1):
    signal[cuts[i]:cuts[i+1]] = signal[cuts[i]:cuts[i+1]]*(np.sqrt(delta[i]))


ss.feed_data(signal)        
result = []
tresult = []
cont = 1
ntotal = len(betalist)*len(alphalist)
for alpha in alphalist:
    for beta in betalist:
        ss.initialize(beta, alpha, mciter, mcburn, nchains = 1)
        t, tdur = ss.segments(minlen = 5000, res = 1, verbose = False)
        result.append([alpha, beta, len(t)+1])
        #t.sort()
        tresult.append(t)

        print('({:.2%})'.format(cont/ntotal) + ' Alpha = {:.3}'.format(alpha) + ', Beta = {:.5}'.format(beta) + ", " + str(len(t)+1) + " segments")
        cont = cont + 1         


